<a href="https://colab.research.google.com/github/petteriTeikari/minivess_mlops/blob/main/notebooks/train_monai_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use a custom model quickly

Reuse the data(loader) from the "ML pipe"

Import the libraries needed

In [27]:
import sys
import os

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


If you run this Notebook on Colab, you need to install the Virtual Environment with Poetry yourself (what I understood):

In [28]:
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    print('You are running this on COLAB so installing the environment here')
    os.chdir("/content")
    !git clone https://github.com/petteriTeikari/minivess_mlops.git
    !pip install poetry
    os.chdir("/content/minivess_mlops")
    !poetry config virtualenvs.in-project true
    # Running "!poetry install --no-ansi" is needed or not?
    !poetry install
    #!poetry shell
    # https://stackoverflow.com/a/65440080/6412152
    !poetry show -v
else:
    print('Assuming that you are runnign this from IDE,\n'
          'or some other environment where you have your Jupyter kernel created from Poetry files')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


You are running this on COLAB so installing the environment here
fatal: destination path 'minivess_mlops' already exists and is not an empty directory.
Installing dependencies from lock file

No dependencies to install or update
Using virtualenv: /root/.cache/pypoetry/virtualenvs/mlops-1FqrDWw7-py3.10
absl-py                   2.0.0        Abseil Python Common Libraries, see https://github.com/a...
aiobotocore               2.5.4        Async client for aws services using botocore and aiohttp
aiohttp                   3.8.6        Async http client/server framework (asyncio)
aiohttp-retry             2.8.3        Simple retry client for aiohttp
aioitertools              0.11.0       itertools and builtins for AsyncIO and mixed iterables
aiosignal                 1.3.1        aiosignal: a list of registered asynchronous callbacks
alembic                   1.12.0       A database migration tool for SQLAlchemy.
amqp                      5.1.1        Low-level AMQP client for Python (fork 

Import the other modules now available from Poetry environment and from the Github repo (see [in_Google_Colab.ipynb](https://github.com/elise-chin/poetry-and-colab/blob/main/Using_python_poetry_in_Google_Colab.ipynb))

In [34]:
VENV_BASE = '/root/.cache/pypoetry/virtualenvs/mlops-1FqrDWw7-py3.10'
VENV_PATH = os.path.join(VENV_BASE, 'lib', 'python3.10', 'site-packages')
#VENV_PATH = "/content/gdrive/MyDrive/test-poetry/.venv/lib/python3.8/site-packages"
#!mkdir /content/venv
LOCAL_VENV_PATH = '/content/venv_poetry' # local notebook
try:
  os.symlink(VENV_PATH, LOCAL_VENV_PATH) # connect to directory in drive
except Exception as e:
  print('symlink failed, e = {}'.format(e))
sys.path.insert(0, LOCAL_VENV_PATH)

sys.path.insert(0,'/content/minivess_mlops')

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


symlink failed, e = [Errno 17] File exists: '/root/.cache/pypoetry/virtualenvs/mlops-1FqrDWw7-py3.10/lib/python3.10/site-packages' -> '/content/venv_poetry'


In [35]:
from loguru import logger

from src.run_training import parse_args_to_dict
from src.training.experiment import define_experiment_data
from src.utils.config_utils import import_config

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


Import helper subfunction(s)

In [36]:
def get_dataloaders(experim_dataloaders: dict):
    # Get the "validation" and "train" dataloaders from the dictionary
    fold_name = 'fold0'
    split_names = list(experim_dataloaders[fold_name].keys())
    fold_key = experim_dataloaders.get(fold_name)
    if fold_key is not None:
        try:
            train = experim_dataloaders[fold_name]['TRAIN']
            val = experim_dataloaders[fold_name]['VAL']['MINIVESS']
        except Exception as e:
            raise IOError('Could not get the dataloaders from the dictionary, error = {}'.format(e))
    else:
        raise IOError('Fold name = "{}" not found in the dataloaders dictionary'.format(fold_name))
    return train, val

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


Input arguments for the training (you can add all the input arguments supported by `run_training.py` here

In [37]:
input_args = ['-c', 'tutorials/train_demo']

# Fake these as coming from the command line to match the main code (run_training.py)
sys.argv = ['notebook_run']  # Jupyter has all the extra crap, so replace that with this
for sysargv in input_args:
    sys.argv.append(sysargv)
args = parse_args_to_dict()

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


2023-10-25 02:25:01.222 | INFO     | src.run_training:parse_args_to_dict:73 - Parsed input arguments:
2023-10-25 02:25:01.224 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   task_config_file: tutorials/train_demo
2023-10-25 02:25:01.226 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   run_mode: train
2023-10-25 02:25:01.228 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   data_dir: /mnt/minivess-dvc-cache
2023-10-25 02:25:01.234 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   output_dir: /mnt/minivess-artifacts
2023-10-25 02:25:01.235 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   s3_mount: False
2023-10-25 02:25:01.236 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   local_rank: 0
2023-10-25 02:25:01.237 | INFO     | src.utils.general_utils:print_dict_to_logger:40 -   project_name: MINIVESS_segmentation_TEST


Create the config with Hydra from the .yaml file(s)

In [38]:
config, exp_run = import_config(args=args, task_cfg_name=args['task_config_file'])

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


2023-10-25 02:25:05.506 | ERROR    | src.utils.config_utils:get_config_path:36 - Configuration dir "/content/minivess_mlops/../configs" does not exist!


OSError: ignored

Import the dataloaders (now the data augmentations are here as well as data transformations)

In [ ]:
_, _, experim_dataloaders, exp_run = (
        define_experiment_data(config=config,
                               exp_run=exp_run))

# Get the "validation" and "train" dataloaders from the dictionary
train, val = get_dataloaders(experim_dataloaders)

Now you are ready to train your new model that you just wanna quickly test without
wanting to have a battle with the config .YAML files

Add maybe some fastai demo with MLflow autologging:
[https://github.com/mlflow/mlflow/blob/master/examples/fastai/train.py](https://github.com/mlflow/mlflow/blob/master/examples/fastai/train.py)

In [ ]:
# Iterate the dataloaders for demo
no_of_epochs = 3
logger.info('Training for {} epochs'.format(no_of_epochs))
for epoch in range(no_of_epochs):

    logger.info('Epoch {}/{}'.format(epoch, no_of_epochs - 1))

    # Train
    logger.info('train with {} batches'.format(len(train)))
    for i, batch in enumerate(train):
        images, mask = batch['image'], batch['label']

    # Validation
    logger.info('validate with {} batches'.format(len(train)))
    for j, batch in enumerate(val):
        images, mask = batch['image'], batch['label']

logger.info('Training done!')